In [1]:
from __future__ import print_function

import numpy as np
import warnings
import tensorflow

%env KERAS_BACKEND=tensorflow
from keras.callbacks import LearningRateScheduler
import keras
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs
from keras.regularizers import l2

def identity_block_preactive(input_tensor, kernel_size,filters):

    filters1, filters2 = filters
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
  
    x = BatchNormalization(axis=bn_axis)(input_tensor)
    x = Activation('relu')(x)
    x = Conv2D(filters1, (3, 3),padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, (3,3),
               padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=l2(0.0001))(x)

    x = layers.add([x, input_tensor])

    return x

def conv_block_preactive(input_tensor, kernel_size, filters, strides=(2, 2)):

    filters1, filters2 = filters
    
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    x_act = Activation('relu')(input_tensor)    
    x = Conv2D(filters1, (3, 3), strides=strides,
                          kernel_initializer="he_normal",
                          padding="same",
                          kernel_regularizer=l2(0.0001))(x_act)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, (3,3), padding='same',
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(x)


    shortcut = Conv2D(filters2, (1, 1), strides=strides,
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(x_act)
    
    x = layers.add([x, shortcut])
    return x

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [59]:
def ResNet_wide(input_tensor=None, input_shape=None,
                pooling=None):
    # Determine proper input shape
    input_shape = (32,32,3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
        
    block_shape = K.int_shape(img_input)
    print(img_input)
    x = Conv2D(16, (3, 3), strides=(1, 1),padding="same")(img_input)
    x = BatchNormalization(axis=bn_axis)(x)
    
    
    block_shape = K.int_shape(x)
    print(block_shape)
    
    x = conv_block_preactive(x, 3, [32,32], strides=(1, 1))
    x = identity_block_preactive(x, 3, [32,32])
    x = identity_block_preactive(x, 3, [32,32]) 
    x = identity_block_preactive(x, 3, [32,32]) 
    x = identity_block_preactive(x, 3, [32,32]) 
    x = identity_block_preactive(x, 3, [32,32]) 
    
    block_shape = K.int_shape(x)
    print(block_shape)
  
    x = BatchNormalization(axis=bn_axis)(x)
    
    x = conv_block_preactive(x, 3, [64,64])
    x = identity_block_preactive(x, 3, [64,64])
    x = identity_block_preactive(x, 3, [64,64])
    x = identity_block_preactive(x, 3, [64,64])
    x = identity_block_preactive(x, 3, [64,64])
    x = identity_block_preactive(x, 3, [64,64])
    block_shape = K.int_shape(x)
    print(block_shape)
    
    x = BatchNormalization(axis=bn_axis)(x)
    
    x = conv_block_preactive(x, 3, [128,128])
    x = identity_block_preactive(x, 3, [128,128])
    x = identity_block_preactive(x, 3, [128,128])
    x = identity_block_preactive(x, 3, [128,128])
    x = identity_block_preactive(x, 3, [128,128])
    x = identity_block_preactive(x, 3, [128,128])
    
    block_shape = K.int_shape(x)
    print(block_shape)
    
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation('relu')(x)
    
    block_shape = K.int_shape(x)
    x = AveragePooling2D(pool_size=(8, 8),strides=(1, 1))(x) 
    x = Flatten()(x)
    x  = Dense(10, activation='softmax', name='fc1000')(x)
    
    block_shape = K.int_shape(x)
    print(block_shape)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')
   
    return model


In [77]:
model = ResNet_wide()

Tensor("input_17:0", shape=(?, 32, 32, 3), dtype=float32)
(None, 32, 32, 16)
(None, 32, 32, 32)
(None, 16, 16, 64)
(None, 8, 8, 128)
(None, 10)


In [78]:
from keras.datasets import cifar10


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255 
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [79]:

from keras.optimizers import SGD

sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])


In [80]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
train_datagen.fit(x_train, augment=True, rounds=2)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
nb_classes = 10
nb_epoch = 200


lr_schedule = [40, 80, 160] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.01 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.001
    return 0.0001

callbacks = [ LearningRateScheduler(schedule=schedule)]
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    validation_data=(x_test, y_test),
                    epochs=nb_epoch,
                    callbacks=callbacks)